# UserProxyAgent using FastAPI

This notebook is a walk-through a modified version of:

- https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_fastapi
To run, open a terminal in Jupyter and execute:


## Running the app

```bash
pip install -r requirements.txt
cd 03_userproxyagent
python app_team.py
```

Open a new browser tab to http://localhost:8002

You should see output in the terminal similar to:

```bash
jovyan@9280d871b652:~/03_userproxyagent$ python app_team.py 
INFO:     Started server process [4400]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)
```

## Terminating the app

Head back to the Jupyter terminal, and press `CTRL-C`.  

You may need to do this a couple of times.  Ignore any error messages.

## Documentation of the app code

This section provides an overview of the application code base in the `work/03_userproxyagent` directory, focusing on the files `app_team.py` and `app_team.html`. Special emphasis is placed on the role of the `UserProxyAgent` within the application.

---

### Overview

The application is a FastAPI-based chat server that hosts a team of agents interacting in a round-robin fashion. The team consists of:

- **AssistantAgent**: A helpful assistant agent.
- **Yoda**: A variant of the assistant agent that repeats messages in the tone of Yoda.
- **UserProxyAgent**: A proxy agent that represents the user, facilitating asynchronous user input via WebSocket.

The front-end interface (`app_team.html`) connects to the backend WebSocket endpoint to send user messages and receive agent responses, enabling real-time chat interaction.

---

### app_team.py

**FastAPI Setup**

- The FastAPI app serves the chat interface HTML (`app_team.html`) at the root endpoint (`/`).
- CORS middleware is configured to allow all origins, methods, and headers.
- Static files are served from the current directory under the `/static` path.

**Team Setup (`get_team` function)**

- Loads model configuration from `model_config.yaml`.
- Instantiates a `ChatCompletionClient` with the loaded config and environment API key.
- Creates three agents:
  - `AssistantAgent` named `"assistant"` with a system message defining its role.
  - `AssistantAgent` named `"yoda"` with a system message to mimic Yoda's tone.
  - `UserProxyAgent` named `"user"` which uses a provided asynchronous input function to receive user input.
- These agents are combined into a `RoundRobinGroupChat` team.
- The team state is loaded from `team_state.json` if it exists.

**UserProxyAgent Role**

- The `UserProxyAgent` acts as a proxy for the human user within the team.
- It is instantiated with an asynchronous input function (`input_func`) that is called whenever the agent requires user input.
- This input function is implemented to asynchronously wait for messages from the WebSocket message queue.
- This design allows the team to request user input asynchronously during the chat flow, enabling interactive conversations.

**WebSocket Chat Endpoint (`/ws/chat`)**

- Accepts WebSocket connections from clients.
- Maintains an asynchronous queue (`message_queue`) to hold incoming user messages.
- Defines an internal `_user_input` async function that the `UserProxyAgent` uses to get user input by awaiting messages from the queue.
- Listens for incoming WebSocket messages, validates them as `TextMessage` objects, and enqueues them.
- Runs the team chat logic in a streaming fashion, sending agent messages back to the client over the WebSocket.
- Saves team state and chat history to JSON files after each interaction.
- Handles disconnections and errors gracefully, sending error messages to the client as needed.

---

### app_team.html

**Front-End Chat Interface**

- A simple HTML page with a chat container displaying messages and an input box with a send button.
- Connects to the backend WebSocket endpoint at `ws://localhost:8002/ws/chat`.
- Sends user messages to the WebSocket when the send button is clicked or Enter is pressed.
- Displays incoming messages from the WebSocket, differentiating between user, assistant, system, and error messages.
- Handles special message types like `UserInputRequestedEvent` to re-enable the input box when the backend requests user input.
- Loads chat history from the backend on page load to display previous messages.

---

### Summary: UserProxyAgent's Role

The `UserProxyAgent` serves as the bridge between the human user and the agent team. It enables the team to request and receive user input asynchronously during the conversation. This is achieved by:

- Passing an asynchronous input function to the `UserProxyAgent` that awaits messages from the WebSocket message queue.
- The WebSocket endpoint receives user messages from the front-end and enqueues them for consumption by the `UserProxyAgent`.
- This design allows the team to operate in a round-robin manner, with the `UserProxyAgent` seamlessly integrating human input into the agent conversation flow.

This architecture enables interactive, real-time chat experiences where AI agents and the user collaborate in a conversational team.


---

In [10]:
from pathlib import Path
from IPython.display import Markdown

def show_file(path: str, lang: str = ""):
    """Display a file in a styled code block in Jupyter without executing it."""
    content = Path(path).read_text()
    display(Markdown(f"Filename: {path}"))
    display(Markdown(f"```{lang}\n{content}\n```"))

# Show the three files
show_file("app_team.py", "python")
show_file("app_team.html", "html")
show_file("model_config.yaml", "yaml")

Filename: app_team.py

```python
import json
import logging
import os
from typing import Any, Awaitable, Callable, Optional

import aiofiles
import yaml
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage, UserInputRequestedEvent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core import CancellationToken
from autogen_core.models import ChatCompletionClient
from fastapi import FastAPI, HTTPException, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles

logger = logging.getLogger(__name__)

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

model_config_path = "model_config.yaml"
state_path = "team_state.json"
history_path = "team_history.json"

# Serve static files
app.mount("/static", StaticFiles(directory="."), name="static")

@app.get("/")
async def root():
    """Serve the chat interface HTML file."""
    return FileResponse("app_team.html")


async def get_team(
    user_input_func: Callable[[str, Optional[CancellationToken]], Awaitable[str]],
) -> RoundRobinGroupChat:
    # Get model client from config.
    async with aiofiles.open(model_config_path, "r") as file:
        model_config = yaml.safe_load(await file.read())
        
    import os
    model_config['config']['api_key'] = os.getenv('OPENAI_API_KEY')
    model_client = ChatCompletionClient.load_component(model_config)
    # Create the team.
    agent = AssistantAgent(
        name="assistant",
        model_client=model_client,
        system_message="You are a helpful assistant.",
    )
    yoda = AssistantAgent(
        name="yoda",
        model_client=model_client,
        system_message="Repeat the same message in the tone of Yoda.",
    )
    user_proxy = UserProxyAgent(
        name="user",
        input_func=user_input_func,  # Use the user input function.
    )
    team = RoundRobinGroupChat(
        [agent, yoda, user_proxy],
    )
    # Load state from file.
    if not os.path.exists(state_path):
        return team
    async with aiofiles.open(state_path, "r") as file:
        state = json.loads(await file.read())
    await team.load_state(state)
    return team


async def get_history() -> list[dict[str, Any]]:
    """Get chat history from file."""
    if not os.path.exists(history_path):
        return []
    async with aiofiles.open(history_path, "r") as file:
        return json.loads(await file.read())


@app.get("/history")
async def history() -> list[dict[str, Any]]:
    try:
        return await get_history()
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e)) from e


@app.websocket("/ws/chat")
async def chat(websocket: WebSocket):
    await websocket.accept()

    import asyncio
    from datetime import datetime

    message_queue: asyncio.Queue[TextMessage] = asyncio.Queue()

    def serialize_datetimes(obj):
        if isinstance(obj, dict):
            return {k: serialize_datetimes(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [serialize_datetimes(i) for i in obj]
        elif isinstance(obj, datetime):
            return obj.isoformat()
        else:
            return obj

    # User input function used by the team.
    async def _user_input(prompt: str, cancellation_token: CancellationToken | None) -> str:
        try:
            message = await message_queue.get()
            return message.content
        except asyncio.CancelledError:
            logger.info("User input cancelled")
            raise
        except Exception as e:
            logger.error(f"Error in user input function: {str(e)}")
            raise

    async def receive_messages():
        try:
            while True:
                data = await websocket.receive_json()
                message = TextMessage.model_validate(data)
                await message_queue.put(message)
        except WebSocketDisconnect:
            logger.info("Client disconnected during message receiving")
        except Exception as e:
            logger.error(f"Error receiving messages: {str(e)}")

    receive_task = asyncio.create_task(receive_messages())

    try:
        while True:
            # Wait for the next user message from the queue
            request = await message_queue.get()

            try:
                # Get the team and respond to the message.
                team = await get_team(_user_input)
                history = await get_history()
                stream = team.run_stream(task=request)
                async for message in stream:
                    if isinstance(message, TaskResult):
                        continue
                    message_dict = serialize_datetimes(message.model_dump())
                    await websocket.send_json(message_dict)
                    if not isinstance(message, UserInputRequestedEvent):
                        # Don't save user input events to history.
                        history.append(message_dict)

                # Save team state to file.
                async with aiofiles.open(state_path, "w") as file:
                    state = await team.save_state()
                    await file.write(json.dumps(state))

                # Save chat history to file.
                async with aiofiles.open(history_path, "w") as file:
                    await file.write(json.dumps(history))

            except WebSocketDisconnect:
                # Client disconnected during message processing - exit gracefully
                logger.info("Client disconnected during message processing")
                break
            except Exception as e:
                # Send error message to client
                error_message = {
                    "type": "error",
                    "content": f"Error: {str(e)}",
                    "source": "system"
                }
                try:
                    await websocket.send_json(error_message)
                    # Re-enable input after error
                    await websocket.send_json({
                        "type": "UserInputRequestedEvent",
                        "content": "An error occurred. Please try again.",
                        "source": "system"
                    })
                except WebSocketDisconnect:
                    # Client disconnected while sending error - exit gracefully
                    logger.info("Client disconnected while sending error message")
                    break
                except Exception as send_error:
                    logger.error(f"Failed to send error message: {str(send_error)}")
                    break

    except WebSocketDisconnect:
        logger.info("Client disconnected")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        try:
            await websocket.send_json({
                "type": "error",
                "content": f"Unexpected error: {str(e)}",
                "source": "system"
            })
        except WebSocketDisconnect:
            # Client already disconnected - no need to send
            logger.info("Client disconnected before error could be sent")
        except Exception:
            # Failed to send error message - connection likely broken
            logger.error("Failed to send error message to client")
            pass
    finally:
        receive_task.cancel()


# Example usage
if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8002)

```

Filename: app_team.html

```html
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AutoGen FastAPI Sample: Team</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            background-color: #f0f0f0;
        }

        #chat-container {
            width: 90%;
            max-width: 600px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }

        #messages {
            height: 600px;
            overflow-y: auto;
            border-bottom: 1px solid #ddd;
            margin-bottom: 20px;
        }

        .message {
            margin: 10px 0;
        }

        .message.user {
            text-align: right;
        }

        .message.assistant {
            text-align: left;
        }

        .label {
            font-weight: bold;
            display: block;
        }

        .content {
            margin-top: 5px;
        }

        #input-container {
            display: flex;
        }

        #input-container input {
            flex: 1;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
        }

        #input-container button {
            padding: 10px 20px;
            border: none;
            background-color: #007bff;
            color: #fff;
            border-radius: 4px;
            cursor: pointer;
        }

        #input-container input:disabled,
        #input-container button:disabled {
            background-color: #e0e0e0;
            cursor: not-allowed;
        }

        .message.error {
            color: #721c24;
            background-color: #f8d7da;
            border: 1px solid #f5c6cb;
            padding: 10px;
            border-radius: 4px;
            margin: 10px 0;
        }

        .message.system {
            color: #0c5460;
            background-color: #d1ecf1;
            border: 1px solid #bee5eb;
            padding: 10px;
            border-radius: 4px;
            margin: 10px 0;
        }
    </style>
</head>

<body>
    <div id="chat-container">
        <div id="messages"></div>
        <div id="input-container">
            <input type="text" id="message-input" placeholder="Type a message...">
            <button id="send-button" onclick="sendMessage()">Send</button>
        </div>
    </div>

    <script>
        const ws = new WebSocket('ws://localhost:8002/ws/chat');

        ws.onmessage = function (event) {
            const message = JSON.parse(event.data);

            if (message.type === 'UserInputRequestedEvent') {
                // Re-enable input and send button if UserInputRequestedEvent is received
                enableInput();
            }
            else if (message.type === 'error') {
                // Display error message
                displayMessage(message.content, 'error');
                enableInput();
            }
            else {
                // Display regular message
                displayMessage(message.content, message.source);
            }
        };

        ws.onerror = function(error) {
            displayMessage("WebSocket error occurred. Please refresh the page.", 'error');
            enableInput();
        };

        ws.onclose = function() {
            displayMessage("Connection closed. Please refresh the page.", 'system');
            disableInput();
        };

        document.getElementById('message-input').addEventListener('keydown', function (event) {
            if (event.key === 'Enter' && !event.target.disabled) {
                sendMessage();
            }
        });

        async function sendMessage() {
            const input = document.getElementById('message-input');
            const button = document.getElementById('send-button');
            const message = input.value;
            if (!message) return;

            // Clear input and disable input and send button
            input.value = '';
            disableInput();

            // Send message to WebSocket
            ws.send(JSON.stringify({ content: message, source: 'user' }));
        }

        function displayMessage(content, source) {
            const messagesContainer = document.getElementById('messages');
            const messageElement = document.createElement('div');
            messageElement.className = `message ${source}`;

            const labelElement = document.createElement('span');
            labelElement.className = 'label';
            labelElement.textContent = source;

            const contentElement = document.createElement('div');
            contentElement.className = 'content';
            contentElement.textContent = content;

            messageElement.appendChild(labelElement);
            messageElement.appendChild(contentElement);
            messagesContainer.appendChild(messageElement);
            messagesContainer.scrollTop = messagesContainer.scrollHeight;
        }

        function disableInput() {
            const input = document.getElementById('message-input');
            const button = document.getElementById('send-button');
            input.disabled = true;
            button.disabled = true;
        }

        function enableInput() {
            const input = document.getElementById('message-input');
            const button = document.getElementById('send-button');
            input.disabled = false;
            button.disabled = false;
        }

        async function loadHistory() {
            try {
                const response = await fetch('http://localhost:8002/history');
                if (!response.ok) {
                    throw new Error('Network response was not ok');
                }
                const history = await response.json();
                history.forEach(message => {
                    displayMessage(message.content, message.source);
                });
            } catch (error) {
                console.error('Error loading history:', error);
            }
        }

        // Load chat history when the page loads
        window.onload = loadHistory;
    </script>
</body>

</html>
```

Filename: model_config.yaml

```yaml
# Use Open AI with key
provider: autogen_ext.models.openai.OpenAIChatCompletionClient
config:
  model: gpt-4o
  api_key: ${OPENAI_API_KEY}

```